In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import random
from scipy.special import rel_entr

In [2]:
################## DATA FRAME ##################
datafile = 'Data/Fitbit_Kaggle/dailyActivity_merged.csv'
all_fields = ['Id', 'ActivityDate', 'TotalSteps', 'TotalDistance', 'TrackerDistance', 
          'VeryActiveDistance', 'ModeratelyActiveDistance', 'LightActiveDistance', 'SedentaryActiveDistance', 
          'VeryActiveMinutes', 'FairlyActiveMinutes', 'LightlyActiveMinutes', 'SedentaryMinutes', 'Calories']

#pd.set_option('display.max_rows', 100)
df = pd.read_csv(datafile, skipinitialspace=True, usecols=all_fields)
fields = 'FairlyActiveMinutes'
selected_data = df[['Id',fields]]
n_rows, n_cols = df.shape

In [3]:
selected_data.head()

,Id,FairlyActiveMinutes
0,1503960366,13
1,1503960366,19
2,1503960366,11
3,1503960366,34
4,1503960366,10


## Generate attack vectors

In [4]:
#['TotalSteps', 'TotalDistance', 'TrackerDistance', 
#          'VeryActiveDistance', 'ModeratelyActiveDistance', 'LightActiveDistance', 
#          'VeryActiveMinutes', 'FairlyActiveMinutes', 'LightlyActiveMinutes', 'SedentaryMinutes', 'Calories']

all_IDs = df.Id.unique()
num_users = len(all_IDs)

user_dict = dict()
for idx, user in enumerate(all_IDs):
    temp_df = df.loc[df['Id'] == user]
    user_dict[user] = temp_df[fields].values

In [28]:
test_IDs = all_IDs[[0, 1]]  # [0, 5, 6, 10, 11, 15, 16, 20, 25, 30]
training_IDs = [ID for ID in all_IDs if ID not in test_IDs]
attack_IDs = list(test_IDs) + list(training_IDs[0:2])

ths = [0]

In [41]:
attack_IDs = [1503960366]
training_IDs = [5577150313, 6117666160, 6290855005, 6775888955]

# Max number of elements for any field
max_elements = 31

data_storage = np.zeros((len(attack_IDs), len(training_IDs), len(ths), max_elements))
accuracy_matrix = np.zeros((len(attack_IDs), len(training_IDs), len(ths)))

In [42]:
for i, attack_ID in enumerate(attack_IDs):
    for j, training_ID in enumerate(training_IDs):
        #for k, training_value in enumerate(user_dict[training_ID]):
        for l, th in enumerate(ths):
            for m, attack_value in enumerate(user_dict[attack_ID]):
                if ((np.array(user_dict[training_ID]) - attack_value) < th).any():
                    # Then this given attack value is in the given person's vector
                    data_storage[i,j,l,m] = 1
                    
            # Once we have established membership for all attack values:
            # Calculate the accuracy of that given vector
            # E.g. percentage score of how many of attack values were present in given person
            accuracy_matrix[i,j,l] = sum(data_storage[i,j,l,:]) / len(user_dict[attack_ID])
            

In [ ]:
attack_IDs = [1503960366]
training_IDs = [5577150313, 6117666160, 6290855005, 6775888955]

In [57]:
set(user_dict[1503960366])

{0,
 5,
 6,
 8,
 10,
 11,
 12,
 13,
 14,
 15,
 16,
 19,
 20,
 21,
 22,
 23,
 24,
 27,
 28,
 31,
 34,
 35,
 46}

In [56]:
set(user_dict[5577150313])

{0,
 8,
 9,
 11,
 15,
 16,
 20,
 21,
 23,
 24,
 30,
 32,
 36,
 37,
 38,
 45,
 51,
 54,
 56,
 65,
 72,
 74}

In [53]:
set(user_dict[6117666160])

{0, 6, 7, 10, 15, 19}

In [54]:
set(user_dict[6290855005])

{0, 3, 12, 95}

In [55]:
set(user_dict[6775888955])

{0, 5, 9, 13, 18, 24, 30, 31, 34, 35, 113}

In [47]:
data_storage

array([[[[1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
          1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
          0.]],

        [[1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
          1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
          0.]],

        [[1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
          1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
          0.]],

        [[1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
          1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
          0.]]]])

In [44]:
accuracy_matrix[:,:,0].shape

(1, 4)

In [45]:
accuracy_matrix[:,:,0]

array([[0.96774194, 0.96774194, 0.96774194, 0.96774194]])

In [46]:
accuracy_matrix[:,:,1]

IndexError: index 1 is out of bounds for axis 2 with size 1

In [10]:
raise("Dont run")

TypeError: exceptions must derive from BaseException

In [ ]:
def isHit(activity, query, ths):
    
    hits = list()
    for th in ths:
        hit = False
        i = 0
        for j in range(0,len(activity)):
            #print(query[j])
            if (abs(query[i] - activity[j])<=th):
                i = i + 1
            
                if i == len(query):
                    hit = True
                    break
        
        hits.append(hit)
    return hits


In [ ]:
def uniqueness(queries, domain, k, l, uniq_score, individuals, ths):
    '''
    queries:
    domain:
    k: limit 
    l: current iteration I think, current number of entries more like
    individuals: individualAct[Id] = act['FairlyActiveMinutes'].tolist()
    '''
    
    print(queries[0])
    print(len(queries))
    print(len(individuals.keys()))
    
    if l > k:
        print("L > K!!!")
        return uniq_score
    
    present_queries = []
    query_matches = dict()
    
    for query in queries:
        match = dict()
        for th in ths:
            match[th] = []
            
        inData = False
        #print(query)
        for Id in individuals:
            ind_act = individuals[Id]
                
            hits = isHit(ind_act, query, ths)
                
            for i in range(0,len(ths)):
                hit = hits[i]
                
                if hit:
                    match[ths[i]].append(Id)
                    inData = True
                    #print(ths[i])
                    #print(Id)
                    #print(match)
        if inData:
            present_queries.append(query)
            query_matches[str(query)] = match
            #print(match)
    
    for i in range(0, len(ths)):
        uniquesId = []
        th = ths[i]
        #print(th)
        
        for keyquery in query_matches:
            match = query_matches[keyquery]
            
            matchTH = match[th]
            #print(keyquery)
            #print(matchTH)
            if len(list(set(matchTH))) == 1:
                #print("Unique")
                uniquesId.append(matchTH[0]) 
        
        score = (1.0*len(list(set(uniquesId))))/len(individuals.keys())
        uniq_score.append((l,th,score))
                        
    l = l + 1
    newqueries = []
    for query in present_queries:  
        for val in domain:
            newquery = query.copy()
            newquery.append(val)
            newqueries.append(newquery)
                   
    return uniqueness(newqueries, domain, k, l, uniq_score, individuals, ths)
            

In [ ]:
#uniqueness up to 3 readings with th=0 and th=2
uniqueScores = list()
start_query = [[a] for a in data_act]
uniqueScores = uniqueness(start_query, data_act, 3, 1, uniqueScores, individualAct, [0, 2, 5])